In [1]:
print('OK')

OK


In [63]:
# importing all require library
from langchain.llms import GooglePalm
from langchain.chat_models import ChatGooglePalm
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
# extract data from PDF

def pdf_loader(data):
    loader=DirectoryLoader(data,
                        glob='*.pdf',
                        loader_cls=PyPDFLoader)

    document=loader.load()

    return document

In [10]:
extracted_data=pdf_loader('./../Data/')

In [14]:
# create text chunks

def text_split(extracted_data):
    text_spliter=RecursiveCharacterTextSplitter(chunk_size=600,chunk_overlap=100)
    text_chunks=text_spliter.split_documents(extracted_data)

    return text_chunks

In [15]:
text_chunks=text_split(extracted_data)
print("lenght of my chunks : ",len(text_chunks))

lenght of my chunks :  5992


In [16]:
# dowmload embedded model
def hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings=hugging_face_embeddings()

d:\End-to-end-medical-chatbot\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
query_result=embeddings.embed_query("Hello world")
print("lenght",len(query_result))

lenght 384


In [20]:
import os

In [24]:
from pinecone import Pinecone,ServerlessSpec

In [25]:
# Initializaing Pinecone
pc=Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

In [26]:
index_name='medical-chatbot'

In [28]:
pc.list_indexes().names()

['medical-chatbot', 'govt-budget-project', 'vecotr-db-test']

In [29]:
# access the index for further 
index=pc.Index(index_name)

In [31]:
from langchain.vectorstores import Pinecone

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [55]:
docsearch=Pinecone.from_existing_index(index_name,embeddings)

In [56]:
query="What is Allergies"

docs=docsearch.similarity_search(query,k=3)

print("Result",docs)

Result [Document(page_content='Richard Robinson\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune sys-\ntem that occur in response to otherwise harmless sub-stances.\nGALE ENCYCLOPEDIA OF MEDICINE 2 114AllergiesGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 114'), Document(page_content='Causes\nAllergic rhinitis is a type of immune reaction. Nor-\nmally, the immune system responds to foreign microor-ganisms, or particles, like pollen or dust, by producingspecific proteins, called antibodies, that are capable ofbinding to identifying molecules, or antigens, on the for-eign particle. This reaction between antibody and antigensets off a series of reactions designed to protect the bodyfrom infection. Sometimes, this same series of reactionsis triggered by harmless, everyday substances. This is thecondition known as allergy, and the offending substanceis called an allergen.'), Document(page_content='mally, the immune system responds to foreign microor-ganisms and partic

In [37]:
prompt_template='''
Use the following following pieces of information to answer the user's question
If you don't know the answer, just say you don't know, don't try to make up any answer

context : {context}
question : {question}

Only return the helpful answer below nothing else.
Helpful Answer : 

'''

In [38]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs={"prompt": PROMPT}

In [64]:
llm=ChatGooglePalm(google_api_key=os.getenv('GOOGLE_API_KEY'),temperature=0.5)

In [67]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)
                                

In [70]:
query1="Tell me about breast cancer"

result=qa(query1)

print("Response : ", result["result"])

Response :  Breast cancer is a type of cancer that starts in the breasts. It is the most common cancer among women, excluding skin cancer. In 2020, an estimated 281,550 new cases of invasive breast cancer were diagnosed in women in the United States, along with 49,290 deaths from the disease.

Breast cancer can occur in both women and men, but it is much more common in women. About 1 in 8 women will develop invasive breast cancer over the course of their lifetime.

The risk of breast cancer increases with age. Most cases are diagnosed in women over the age of 50. However, breast cancer can occur in women of any age, even teenagers.

There are a number of factors that can increase the risk of breast cancer, including:

* Age: The risk of breast cancer increases with age. Most cases are diagnosed in women over the age of 50.
* Family history: Women who have a family history of breast cancer are at an increased risk of developing the disease.
* Personal history: Women who have had breast 

In [71]:
query2="what should i do to prevent AIDS"

result=qa(query2)

print("Response : ", result["result"])

Response :  Here are some things you can do to prevent AIDS:

* **Be monogamous and practice safe sex.** This means having sexual intercourse only with one partner who is not infected with HIV. If you do have sex with someone who is not your partner, use a condom every time.
* **Avoid needle sharing among intravenous drug users.** If you inject drugs, use a clean needle and syringe every time. You can also get help to stop using drugs.
* **Get tested for HIV.** If you think you may have been exposed to HIV, get tested as soon as possible. Early diagnosis and treatment can help you live a long and healthy life.
* **If you are pregnant, get tested for HIV.** If you are HIV-positive, you can take steps to prevent your baby from getting infected.
* **Talk to your doctor about HIV prevention.** Your doctor can help you understand the risks of HIV and how to protect yourself.

It is important to remember that there is no cure for AIDS, but there are treatments that can help people with HIV l